In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

from textblob import TextBlob, Word

In [2]:
movies_df=pd.read_csv('./imdb5000/tmdb_5000_movies.csv',low_memory=False)
credits_df=pd.read_csv('./imdb5000/tmdb_5000_credits.csv')
# ratings_df=pd.read_csv('./imdb5000/ratings.csv')

# for index,row in movies_df.iterrows():
# 	try:
# 		row["id"]=int(row["id"])
# 	except:
# 		movies_df.drop(index,axis=0,inplace=True)
# credits_df["id"]=credits_df["id"].astype(object)
# movies_df["id"]=pd.to_numeric(movies_df["id"])
movies_copy0=pd.merge(movies_df,credits_df,how='left',right_on='movie_id',left_on='id')

In [3]:
c=movies_copy0["vote_average"].mean()
m=movies_copy0["vote_count"].shape[0]
print(m)
# movie_list=movies_copy0.copy().loc[movies_copy0['vote_count']>=m
movie_list=movies_copy0.copy()

4803


In [4]:
def rating(x,M=m,C=c):
    v=x['vote_count']
    r=x['vote_average']
    return (r*v + c*m)/(v+m)

In [5]:
movie_list['score']=movie_list.apply(rating,axis=1)

In [6]:
def get_val(row):
    dic=literal_eval(row)
    lst=[]
    count=0
    for d in dic:
        count+=1
        lst.append(d['name'])
        if (count==max):
            break
    return lst

def get_valmax(row):
    dic=literal_eval(row)
    lst=[]
    for d in dic:
        lst.append(d['name'])
    return lst

    # return [d['name'] for d in dic]

In [7]:
def get_dir(row):
    lst=[]
    dic=literal_eval(row)
    for i in dic:
        if i['job']=='Director':
            lst.append(i['name'])
            break
    return lst

In [8]:
movie_list['Director']=list(movie_list['crew'].apply(get_dir))

In [9]:
movie_list['cast']=movie_list['cast'].apply(get_val)
movie_list['genres']=movie_list['genres'].apply(get_val)

In [10]:
movie_list['keywords']=movie_list['keywords'].apply(get_valmax)

In [11]:
movie_list['overview']=movie_list['overview'].fillna('')

In [12]:
movie_list['overview']=movie_list['overview'].apply(lambda x:x.split())

In [13]:
movie_list['Director']=movie_list['Director'].fillna('')

In [14]:
movie_list["keywords"][1]


['ocean',
 'drug abuse',
 'exotic island',
 'east india trading company',
 "love of one's life",
 'traitor',
 'shipwreck',
 'strong woman',
 'ship',
 'alliance',
 'calypso',
 'afterlife',
 'fighter',
 'pirate',
 'swashbuckler',
 'aftercreditsstinger']

In [15]:
movie_list['cast'] = movie_list['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movie_list['genres'] = movie_list['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movie_list['keywords'] = movie_list['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movie_list['Director']=movie_list['Director'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
movie_list['cast'][1]

['JohnnyDepp',
 'OrlandoBloom',
 'KeiraKnightley',
 'StellanSkarsgård',
 'ChowYun-fat',
 'BillNighy',
 'GeoffreyRush',
 'JackDavenport',
 'KevinMcNally',
 'TomHollander',
 'NaomieHarris',
 'JonathanPryce',
 'KeithRichards',
 'LeeArenberg',
 'MackenzieCrook',
 'GregEllis',
 'DavidBailie',
 'MartinKlebba',
 'DavidSchofield',
 'LaurenMaher',
 'VanessaBranch',
 'AngusBarnett',
 'GilesNew',
 'ReggieLee',
 'DominicScottKay',
 'TakayoFischer',
 'DavidMeunier',
 'Ho-KwanTse',
 'AndyBeckwith',
 'PeterDonaldBadalamentiII',
 'ChristopherS.Capp',
 'KeithRichards',
 'HakeemKae-Kazim',
 'GhassanMassoud']

In [17]:
def create_fet(x):
    return ' '.join(x['overview'])+' '+' '.join(x['genres'])+' '.join(x['keywords'])+' '.join(x['cast'])+' '+" ".join(x['Director'])

In [18]:
movie_list['tags']=movie_list.apply(create_fet,axis=1)

In [19]:
movie_list['tags']=movie_list['tags'].apply(lambda x:x.lower())

In [20]:
tfdif=CountVectorizer(stop_words="english")

In [21]:
lemmatizer = WordNetLemmatizer()
ps=PorterStemmer()

In [22]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [23]:
def lemmatizerfunc(sentence):
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])

In [24]:
def stemmer(sen):
    lst=[]
    for i in sen.split():
        lst.append(ps.stem(i))
    return " ".join(lst)

In [25]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [26]:
movie_list['l1']=movie_list['tags'].apply(lemmatizerfunc)
movie_list['l2']=movie_list['tags'].apply(stemmer)
movie_list['l3']=movie_list['tags'].apply(lemmatize_with_postag)

In [27]:
vec1=tfdif.fit_transform(movie_list['l1']).toarray()
vec2=tfdif.fit_transform(movie_list['l2']).toarray()
vec3=tfdif.fit_transform(movie_list['l3']).toarray()

In [28]:
movie_list.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,movie_id,title_y,cast,crew,score,Director,tags,l1,l2,l3
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",6.879522,[JamesCameron],"in the 22nd century, a paraplegic marine is di...","in the 22nd century , a paraplegic marine be d...","in the 22nd century, a parapleg marin is dispa...",in the 22nd century a paraplegic marine be dis...
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drugabuse, exoticisland, eastindiatrad...",en,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",6.482930,[GoreVerbinski],"captain barbossa, long believed to be dead, ha...","captain barbossa , long believe to be dead , h...","captain barbossa, long believ to be dead, ha c...",captain barbossa long believe to be dead have ...
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",en,Spectre,"[A, cryptic, message, from, Bond’s, past, send...",107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,206647,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",6.192308,[SamMendes],a cryptic message from bond’s past sends him o...,a cryptic message from bond ’ s past sends him...,a cryptic messag from bond’ past send him on a...,a cryptic message from bond ’ s past sends him...
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dccomics, crimefighter, terrorist, secretiden...",en,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Anne...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",7.079323,[ChristopherNolan],following the death of district attorney harve...,follow the death of district attorney harvey d...,follow the death of district attorney harvey d...,follow the death of district attorney harvey d...
4,260000000,"[Action, Adventure, ScienceFiction]",http://movies.disney.com/john-carter,49529,"[basedonnovel, mars, medallion, spacetravel, p...",en,John Carter,"[John, Carter, is, a, war-weary,, former, mili...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",6.094572,[AndrewStanton],"john carter is a war-weary, former military ca...","john carter be a war-weary , former military c...","john carter is a war-weary, former militari ca...",john carter be a war-weary former military cap...


In [29]:
movie_list['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefictioncultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3dsamworthington zoesaldana sigourneyweaver stephenlang michellerodriguez giovanniribisi joeldavidmoore cchpounder wesstudi lazalonso dileeprao mattgerald seananthonymoran jasonwhyte scottlawrence kellykilgour jamespatrickpitt seanpatrickmurphy peterdillon kevindorman kelsonhenderson davidvanhorn jacobtomuri michaelblain-rozgay joncurry lukehawker woodyschultz petermensah soniayee jahnelcurfman ilramchoi kylawarren lisaroumain debrawilson chrismala taylorkibby jodielandau julielamm cullenb.madden josephbradymadden frankietorres austinwilson sarawilson tamicawashington-miller lucybriant nathanmei

In [30]:
movie_list['l2'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefictioncultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3dsamworthington zoesaldana sigourneyweav stephenlang michellerodriguez giovanniribisi joeldavidmoor cchpounder wesstudi lazalonso dileeprao mattgerald seananthonymoran jasonwhyt scottlawr kellykilgour jamespatrickpitt seanpatrickmurphi peterdillon kevindorman kelsonhenderson davidvanhorn jacobtomuri michaelblain-rozgay joncurri lukehawk woodyschultz petermensah soniaye jahnelcurfman ilramchoi kylawarren lisaroumain debrawilson chrismala taylorkibbi jodielandau julielamm cullenb.madden josephbradymadden frankietorr austinwilson sarawilson tamicawashington-mil lucybri nathanmeist gerryblair matthewchamberlain paulyat wraywi

In [31]:
indexcs=pd.Series(movie_list.index,index=movie_list['title_x'])

In [32]:
def getrec_overview(target_movie:str,mat):
    movie_titles=movie_list['title_x'].values
    if target_movie not in movie_titles:
        print(target_movie,"Movie not Recognized")
        return
    
    sims=mat
    idx=indexcs[target_movie]
    index_of_target_movie=np.where(movie_titles==target_movie)[0]
    simsc=list(enumerate(sims[idx]))
    
    top10=sorted(simsc,key=lambda x:x[1],reverse=True)[1:11]
    print("Movies similar to ",target_movie," based on ")
    movieidx=[i[0] for i in top10]
    return movie_list['title_x'].iloc[movieidx]

In [33]:
sim1= cosine_similarity(vec1)
sim2= cosine_similarity(vec2)
sim3= cosine_similarity(vec3)

In [34]:
def recommend(movie,similarity):
    movie_titles=movie_list['title_x'].values
    if movie not in movie_titles:
        print(movie,"Movie not Recognized")
        return
    index = movie_list[movie_list['title_x'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(movie_list.iloc[i[0]].title_x)

In [35]:
recommend("Avatar",sim2)

Aliens vs Predator: Requiem
Predators
Beowulf
Aliens
Predator


In [36]:
recommend("Avatar",sim3)

Predator
Aliens
Aliens vs Predator: Requiem
Predators
U.F.O.


In [37]:
recommend("Avatar 2",sim1)

Avatar 2 Movie not Recognized


In [38]:
recommend("Batman Begins",sim1)

The Dark Knight
Batman & Robin
Batman Returns
Batman
The Dark Knight Rises


In [39]:
recommend("Fight Club",sim)

NameError: name 'sim' is not defined

In [ ]:
getrec_overview("Batman Begins",sim1)

Movies similar to  Batman Begins  based on 


65                   The Dark Knight
210                   Batman & Robin
428                   Batman Returns
1359                          Batman
3              The Dark Knight Rises
299                   Batman Forever
238     Teenage Mutant Ninja Turtles
4638        Amidst the Devil's Wings
252                 The Green Hornet
3942             Hobo with a Shotgun
Name: title_x, dtype: object

In [ ]:
movie_list['Director'][0]

['JamesCameron']

In [40]:
import pymongo
import json

json_data = movie_list.to_json(orient="records")

# Parse the JSON string to a list of dictionaries
records = json.loads(json_data)

# Connect to MongoDB and get the collection
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Movies"]
collection = db["details"]

# Insert the records into the collection
collection.insert_many(records)
